<a href="https://colab.research.google.com/github/Jih00nJung/assignment_list/blob/main/system_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. 필수 라이브러리 설치
!pip install requests torch transformers faiss-cpu
!pip install tqdm # 진행률 표시바

In [5]:
import requests
import json
import torch
import numpy as np
import faiss
import pickle
import time
from transformers import BertModel, BertTokenizer, AutoTokenizer, AutoModel
from tqdm import tqdm

# --- 1. 설정 (API 키 입력) ---
# 'sample' 키는 하루 요청 횟수나 데이터 건수에 제한이 있을 수 있습니다.
# 공공데이터포털에서 발급받은 본인의 인증키를 넣는 것을 권장합니다.
API_KEY = "ce4c9a9035f643bb9dba"  # 여기에 발급받은 API KEY 입력
SERVICE_ID = "COOKRCP01" # 조리식품 레시피 서비스 ID
TYPE = "json" # xml 대신 json 사용 (파이썬 친화적)

# --- 2. 데이터 수집 함수 (API) ---
def fetch_all_recipes():
    all_documents = []

    # 한 번에 요청할 개수 (최대 1000개까지 가능)
    step = 1000
    start_idx = 1
    end_idx = step

    print("API로부터 레시피 데이터 수집을 시작합니다...")

    while True:
        # URL 생성 (start/end 인덱스 방식)
        url = f"http://openapi.foodsafetykorea.go.kr/api/{API_KEY}/{SERVICE_ID}/{TYPE}/{start_idx}/{end_idx}"

        try:
            response = requests.get(url)
            data = response.json()

            # API 응답 코드 확인
            try:
                code = data[SERVICE_ID]['RESULT']['CODE']
                total_count = int(data[SERVICE_ID]['total_count']) # 전체 데이터 개수
            except KeyError:
                # 데이터가 없거나 에러인 경우 (sample 키 사용 시 자주 발생)
                print(f"응답 구조가 예상과 다릅니다: {data}")
                break

            if code != 'INFO-000':
                print(f"API 호출 종료 (Code: {code})")
                break

            rows = data[SERVICE_ID]['row']
            if not rows:
                break

            for row in rows:
                # 필요한 필드 추출
                menu_name = row.get('RCP_NM', '이름 없음')    # 메뉴명
                ingredients = row.get('RCP_PARTS_DTLS', '') # 재료 정보

                # 조리 순서 합치기 (MANUAL01 ~ MANUAL20)
                manual_text = ""
                for i in range(1, 21):
                    step_key = f'MANUAL{i:02d}'
                    step_desc = row.get(step_key, '')
                    if step_desc:
                        # 줄바꿈 문자 제거하고 붙이기
                        manual_text += step_desc.replace('\n', ' ').strip() + " "

                # [중요] RAG 검색을 위해 하나의 텍스트 덩어리로 만듦
                # 재료 기반 추천을 원하신다면 재료 정보를 앞쪽에 배치하는 것이 좋습니다.
                full_text = f"요리명: {menu_name}\n재료: {ingredients}\n조리법: {manual_text}"
                all_documents.append(full_text)

            print(f"{start_idx} ~ {end_idx} 번째 데이터 수집 완료 (누적 {len(all_documents)}건 / 전체 {total_count}건)")

            # 다음 페이지 준비
            start_idx += step
            end_idx += step

            # 전체 개수를 넘으면 종료
            if start_idx > total_count:
                break

            # 서버 부하 방지용 딜레이
            time.sleep(0.5)

        except Exception as e:
            print(f"오류 발생: {e}")
            break

    print(f"총 {len(all_documents)}개의 레시피 수집 완료!")
    return all_documents

# --- 3. 실행: 데이터 수집 ---
documents = fetch_all_recipes()

# (테스트용) 데이터가 너무 적으면 샘플 데이터 추가 (sample 키 제한 때문)
if len(documents) < 5:
    print("⚠️ 데이터가 너무 적어 테스트용 데이터를 추가합니다.")
    documents.extend([
        "요리명: 김치찌개\n재료: 김치, 돼지고기, 두부, 파\n조리법: 김치를 볶다가 물을 붓고 끓입니다.",
        "요리명: 된장찌개\n재료: 된장, 애호박, 두부, 감자\n조리법: 멸치 육수에 된장을 풀고 야채를 넣습니다."
    ])

# --- 4. KoBERT 임베딩 및 FAISS 인덱스 생성 ---
print("KoBERT 모델 로딩 중...")
# skt/kobert-base-v1 대신 호환성이 좋은 kobert-tokenizer 사용 권장 방식
# 여기서는 가장 에러가 적은 transformer 기본 방식으로 진행
MODEL_NAME = "skt/kobert-base-v1"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

# GPU 사용 설정 (Colab에서는 GPU 써야 빠릅니다)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

def get_embedding(text_list):
    embeddings = []
    # 배치 단위 처리 (메모리 관리)
    batch_size = 32

    for i in range(0, len(text_list), batch_size):
        batch_texts = text_list[i:i+batch_size]
        inputs = tokenizer(
            batch_texts,
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=256 # 레시피는 기니까 길이를 좀 넉넉하게
        ).to(device)

        # token_type_ids가 문제를 일으킬 수 있으므로 제거합니다.
        # 단일 텍스트 임베딩에서는 일반적으로 필요하지 않습니다.
        if 'token_type_ids' in inputs: # token_type_ids 키가 있는 경우에만 삭제
            del inputs['token_type_ids']

        with torch.no_grad():
            outputs = model(**inputs)

        # [CLS] 토큰 벡터 추출
        cls_emb = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        embeddings.append(cls_emb)

    return np.vstack(embeddings)

print("데이터 벡터화 진행 중 (시간이 좀 걸립니다)...")
embeddings = get_embedding(documents)

print("FAISS 인덱스 생성 중...")
dimension = 768 # KoBERT 출력 차원
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# --- 5. 파일 저장 ---
# 이 두 파일을 다운로드해서 로컬 Docker 폴더에 넣으세요
faiss.write_index(index, "project_data.index")
with open("project_data.pkl", "wb") as f:
    pickle.dump(documents, f)

print("✅ 모든 작업 완료! 'project_data.index'와 'project_data.pkl' 파일을 다운로드하세요.")


API로부터 레시피 데이터 수집을 시작합니다...
1 ~ 1000 번째 데이터 수집 완료 (누적 1000건 / 전체 1146건)
1001 ~ 2000 번째 데이터 수집 완료 (누적 1146건 / 전체 1146건)
총 1146개의 레시피 수집 완료!
KoBERT 모델 로딩 중...
데이터 벡터화 진행 중 (시간이 좀 걸립니다)...
FAISS 인덱스 생성 중...
✅ 모든 작업 완료! 'project_data.index'와 'project_data.pkl' 파일을 다운로드하세요.
